# Conversión de los datos para utilizarlos en el modelo

Hay que aplicar algunas transformaciones al conjunto de datos para poder utilizarlos. Las variables que tenemos son las siguientes:

A: Altitud. Variable numérica que representa la elevación del terreno respecto al nivel del mar.


AC: Anchura de calles y otros espacios abiertos. Variable categórica que se transformó en numérica mediante la asignación de valores medios por rango.


B: Indica si una celda corresponde a un barranco (valor 0) o no (valor 1). Variable binaria.


CICCB: Capacidad de inundación según la curvatura del cauce del barranco. Variable numérica que estima la acumulación de agua en función de la curvatura del terreno.


CIB: Capacidad de inundación del barranco considerando su sección y situación. Variable numérica que refleja la influencia conjunta de la profundidad y anchura del cauce.


CUS: Cobertura y uso del suelo. Variable categórica con los siguientes niveles:


1: Cultivos


2: Edificios de cualquier uso


3: Espacios de suelo desnudo


4: Espacios verdes urbanos


5: Sistema vial


DBB: Distancia al borde del cauce del barranco. Variable numérica, medida en metros.


DOB: Distancia a obstáculos en el cauce del barranco. Variable numérica, también en metros.


OC: Orientación de calles y otros espacios abiertos. Variable categórica con los siguientes niveles:


1: Oblicua


2: Paralela


3: Perpendicular


4: Otras


P: Pendiente del terreno. Variable numérica expresada en porcentaje, que representa la inclinación del terreno.


En el conjunto de variables que tenemos, existe una, B, que indica si los datos obtenidos provienen del barranco o no. Los datos que pertenezcan al barranco no sirven, por lo tanto hay que proceder a eliminar todas las filas en las que B = 1.

In [1]:
import pandas as pd
import numpy as np


data = pd.read_csv("D:/HUGO/MAtriX/datos_combinados.csv")

# Vemos cuántas filas hay con 'B' y sin 'B'
print("Conteo de valores en la columna 'B':")
print(data['B'].value_counts(dropna=False)) # dropna=False para incluir conteo de NaNs si los hay

# Eliminamos las filas donde 'B' sea igual a 1 y luego quitamos 'B' de data
# Filtrar filas: data[data['B'] != 1] selecciona todas las filas donde 'B' no es 1
# Eliminar columna: .drop('B', axis=1) elimina la columna 'B'. axis=1 indica que es una columna.
data = data[data['B'] != 1].drop('B', axis=1)

print("\nPrimeras filas del DataFrame después de filtrar y eliminar la columna 'B':")
print(data.head())

Conteo de valores en la columna 'B':
0    64860
1     2352
Name: B, dtype: int64

Primeras filas del DataFrame después de filtrar y eliminar la columna 'B':
         POINT_X       POINT_Y  PLA  grid_cod          A  AC          CIB  \
0  722404.251470  4.365852e+06  0.0      6909  17.955999  11 -9999.000000   
1  722396.507981  4.365860e+06  0.0      7246  18.179501   4     0.271946   
2  722404.245816  4.365860e+06  0.0      7247  18.274900  11     0.269777   
3  722411.983652  4.365860e+06  0.0      7248  18.229401  11     0.267608   
4  722419.721567  4.365860e+06  0.0      7249  17.900000  11 -9999.000000   

      CICCB  CUS         DBB          DOB  OC         P  
0  0.448429    1  848.570984     0.000000   4  4.771400  
1  0.450929    5  855.966980     0.000000   4  1.352900  
2  0.449347    1  847.986023  1110.060059   4  0.436232  
3  0.447738    1  842.000000  1102.219971   4  2.990650  
4  0.446101    1  834.018982  1094.489990   4  8.666000  


Algunas variables tienen como valor -9999, esto es un valor que no aporta nada al modelo y hay que eliminar las filas que contengan esos valores. Luego obtenemos la cantidad de filas eliminadas, para ver si es un número grande que afectaría mucho al modelo.

In [2]:
# Almacenar la longitud inicial del DataFrame
len_antes_eliminar = len(data)

# Eliminar filas que contengan el valor -9999 en cualquier columna
# data == -9999 crea un DataFrame booleano (True donde hay -9999)
# .any(axis=1) verifica si alguna columna en esa fila es True (es decir, tiene -9999)
# ~ niega la máscara, seleccionando las filas que NO tienen -9999
data = data[~(data == -9999).any(axis=1)]

# Almacenar la longitud final
len_despues_eliminar = len(data)

# Mostrar la cantidad de filas eliminadas
print(f"Filas eliminadas: {len_antes_eliminar - len_despues_eliminar}")

Filas eliminadas: 450


Ahora sustituímos las categorías de AC (anchura de calles) por el valor medio de cada rango.

In [3]:
# Hacemos la lista con las medias, y ponemos 0 al principio porque python cuenta desde 0
medias= [0, 3, 9.05, 14.05, 18.05, 25.55, 35.05, 45.05, 75.05, 110.05, 135.05, 225.05] 

# Convertimos la lista de medias a un array de NumPy para permitir el indexado avanzado
medias_array = np.array(medias)

# Realiza la sustitución:
data['AC'] = medias_array[data['AC']]

# Comprobar que las categorías corresponden (ahora son los valores medios)
print("\nConteo de valores en la columna 'AC' después de la sustitución:")
print(data['AC'].value_counts(dropna=False))

# Verificar el tipo de dato
print("\nTipo de dato de la columna 'AC' después de la sustitución:")
print(data['AC'].dtype)


Conteo de valores en la columna 'AC' después de la sustitución:
225.05    18936
75.05     12089
18.05      5560
45.05      5506
35.05      5472
25.55      4851
135.05     3871
14.05      3739
110.05     2103
9.05       2101
3.00        182
Name: AC, dtype: int64

Tipo de dato de la columna 'AC' después de la sustitución:
float64


In [4]:
# Transformar variables OC y CUS en categóricas
# Usamos .astype('category') para convertir columnas a tipo categórico
data['CUS'] = data['CUS'].astype('category')
data['OC'] = data['OC'].astype('category')

# Verificar la estructura del DataFrame (equivalente a str(data) en R)
print(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64410 entries, 1 to 67210
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   POINT_X   64410 non-null  float64 
 1   POINT_Y   64410 non-null  float64 
 2   PLA       64410 non-null  float64 
 3   grid_cod  64410 non-null  int64   
 4   A         64410 non-null  float64 
 5   AC        64410 non-null  float64 
 6   CIB       64410 non-null  float64 
 7   CICCB     64410 non-null  float64 
 8   CUS       64410 non-null  category
 9   DBB       64410 non-null  float64 
 10  DOB       64410 non-null  float64 
 11  OC        64410 non-null  category
 12  P         64410 non-null  float64 
dtypes: category(2), float64(10), int64(1)
memory usage: 6.0 MB
None


Ahora utilizaremos la columna grid_cod, que es un código que se le da a cada celda, como índice para identificarlas.

In [5]:
# Verificar si grid_cod es único
if data['grid_cod'].nunique() == len(data):
    # Si es único, usarlo como índice y eliminar la columna
    data = data.set_index('grid_cod')
    print("grid_cod es único y se ha usado como índice.")
else:
    # Si no es único, eliminarlo directamente
    # axis=1 indica que se elimina una columna
    data = data.drop('grid_cod', axis=1)
    print("grid_cod no es único y se ha eliminado.")

# Mostrar las primeras filas del dataset transformado
print("\nPrimeras filas del DataFrame transformado:")
print(data.head())

grid_cod es único y se ha usado como índice.

Primeras filas del DataFrame transformado:
                POINT_X       POINT_Y  PLA          A      AC       CIB  \
grid_cod                                                                  
7246      722396.507981  4.365860e+06  0.0  18.179501   18.05  0.271946   
7247      722404.245816  4.365860e+06  0.0  18.274900  225.05  0.269777   
7248      722411.983652  4.365860e+06  0.0  18.229401  225.05  0.267608   
7584      722396.502336  4.365868e+06  0.0  18.300400  225.05  0.272302   
7585      722404.240164  4.365868e+06  0.0  18.286400  225.05  0.270117   

             CICCB CUS         DBB          DOB OC         P  
grid_cod                                                      
7246      0.450929   5  855.966980     0.000000  4  1.352900  
7247      0.449347   1  847.986023  1110.060059  4  0.436232  
7248      0.447738   1  842.000000  1102.219971  4  2.990650  
7584      0.451874   1  855.581970     0.000000  4  0.801976  
7585   

Hay que convertir la pendiente a porcentaje

In [6]:
# Convertir grados a radianes
data['P'] = (np.tan(data['P'] * (np.pi / 180))) * 100
print(data.head())

                POINT_X       POINT_Y  PLA          A      AC       CIB  \
grid_cod                                                                  
7246      722396.507981  4.365860e+06  0.0  18.179501   18.05  0.271946   
7247      722404.245816  4.365860e+06  0.0  18.274900  225.05  0.269777   
7248      722411.983652  4.365860e+06  0.0  18.229401  225.05  0.267608   
7584      722396.502336  4.365868e+06  0.0  18.300400  225.05  0.272302   
7585      722404.240164  4.365868e+06  0.0  18.286400  225.05  0.270117   

             CICCB CUS         DBB          DOB OC         P  
grid_cod                                                      
7246      0.450929   5  855.966980     0.000000  4  2.361695  
7247      0.449347   1  847.986023  1110.060059  4  0.761383  
7248      0.447738   1  842.000000  1102.219971  4  5.224414  
7584      0.451874   1  855.581970     0.000000  4  1.399804  
7585      0.450287   1  847.596985  1110.060059  4  0.606890  


Ahora guardamos el resultado

In [7]:
# Guardar el resultado en un nuevo CSV
nombre_archivo_salida = "data_stored.csv"
data.to_csv(nombre_archivo_salida, index=False) # index=False evita guardar el índice de pandas como una columna

print(f"\nDataFrame combinado guardado exitosamente como '{nombre_archivo_salida}'")


DataFrame combinado guardado exitosamente como 'data_stored.csv'
